# Creating DATABASE

In [42]:
import sqlite3
import pandas as pd

# Paths to your cleaned CSV files
solar_file = "advanced_enhanced_solar_company_data.csv"
combined_stats_file = "combined_company_statistics.csv"
indicators_file = "final_indicators_commodities_macro_data.csv"
oil_file = "final_oil_company_data.csv"
wind_file = "final_wind_company_data.csv"

# Create or connect to an SQLite database
db_path = "financial_data.db"
conn = sqlite3.connect(db_path)

# Function to save a CSV to an SQLite table
def save_to_sqlite(csv_file, table_name, conn):
    data = pd.read_csv(csv_file)
    data.to_sql(table_name, conn, if_exists='replace', index=False)
    print(f"Data from {csv_file} saved to table '{table_name}'.")

# Save each dataset to its respective table
save_to_sqlite(solar_file, "solar_data", conn)
save_to_sqlite(combined_stats_file, "combined_statistics", conn)
save_to_sqlite(indicators_file, "indicators_data", conn)
save_to_sqlite(oil_file, "oil_data", conn)
save_to_sqlite(wind_file, "wind_data", conn)

# Close the connection
conn.close()
print(f"All data has been saved to {db_path}.")



Data from advanced_enhanced_solar_company_data.csv saved to table 'solar_data'.
Data from combined_company_statistics.csv saved to table 'combined_statistics'.
Data from final_indicators_commodities_macro_data.csv saved to table 'indicators_data'.
Data from final_oil_company_data.csv saved to table 'oil_data'.
Data from final_wind_company_data.csv saved to table 'wind_data'.
All data has been saved to financial_data.db.


In [44]:
from google.colab import files
files.download("financial_data.db")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [45]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect("financial_data.db")

# Query a specific table (example: solar_data)
pd.read_sql("SELECT * FROM wind_data LIMIT 5;", conn)

pd.read_sql("SELECT * FROM wind_data LIMIT 5;", conn)
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, conn)
print("Available tables:", tables)

# Close the connection
conn.close()


Available tables:                   name
0           solar_data
1  combined_statistics
2      indicators_data
3             oil_data
4            wind_data


In [47]:
# Query to fetch data for 2024-12-20 across all relevant tables
query = """
WITH ranked_data AS (
    SELECT
        *,
        ROW_NUMBER() OVER (PARTITION BY Asset ORDER BY Date DESC) AS row_num
    FROM (
        SELECT Date, Close, High, Low, Open, Volume, Asset FROM solar_data
        UNION ALL
        SELECT Date, Close, High, Low, Open, Volume, Asset FROM indicators_data
        UNION ALL
        SELECT Date, Close, High, Low, Open, Volume, Asset FROM oil_data
        UNION ALL
        SELECT Date, Close, High, Low, Open, Volume, Asset FROM wind_data
    )
)
SELECT *
FROM ranked_data
WHERE Date = '2024-12-20';
"""

# Connect to the SQLite database
conn = sqlite3.connect("financial_data.db")

# Execute the query and fetch the data
try:
    result_df = pd.read_sql(query, conn)
    print("Data for 2024-12-20 across all relevant tables:")
    print(result_df.head())
except Exception as e:
    print(f"Error fetching data for 2024-12-20: {e}")

# Close the database connection
conn.close()



Data for 2024-12-20 across all relevant tables:
         Date         Close          High           Low          Open  \
0  2024-12-20     28.600000     28.740000     28.139999     28.209999   
1  2024-12-20  97755.929688  98098.914062  92175.179688  97484.695312   
2  2024-12-20     69.459999     69.849998     68.419998     69.230003   
3  2024-12-20     11.500000     11.650000     10.600000     10.670000   
4  2024-12-20    142.850006    142.850006    140.550003    140.820007   

         Volume    Asset  row_num  
0  1.275210e+07       BP        7  
1  1.056341e+11  BTC-USD       12  
2  2.315920e+05     CL=F        7  
3  2.360900e+06     CSIQ        7  
4  3.220920e+07      CVX        7  


# Fetching the Schema of the Database

In [48]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
db_path = "financial_data.db"
conn = sqlite3.connect(db_path)

# Query to list all tables
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(tables_query, conn)

print("Available tables in the database:")
print(tables)

# Loop through each table and fetch its schema
print("\nSchema for each table:")
for table in tables['name']:
    print(f"\nSchema for table: {table}")
    schema_query = f"PRAGMA table_info({table});"
    schema = pd.read_sql(schema_query, conn)
    print(schema)

# Close the connection
conn.close()


Available tables in the database:
                  name
0           solar_data
1  combined_statistics
2      indicators_data
3             oil_data
4            wind_data

Schema for each table:

Schema for table: solar_data
    cid                name     type  notnull dflt_value  pk
0     0                Date     TEXT        0       None   0
1     1               Close     REAL        0       None   0
2     2                High     REAL        0       None   0
3     3                 Low     REAL        0       None   0
4     4                Open     REAL        0       None   0
5     5              Volume     REAL        0       None   0
6     6               Asset     TEXT        0       None   0
7     7                Type     TEXT        0       None   0
8     8         Daily_Range     REAL        0       None   0
9     9    Normalized_Range     REAL        0       None   0
10   10        Daily_Return     REAL        0       None   0
11   11                MA_7     REAL      

In [49]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
db_path = "financial_data.db"
conn = sqlite3.connect(db_path)

# Tables to update
tables_to_update = ["solar_data", "indicators_data", "oil_data", "wind_data"]

for table in tables_to_update:
    print(f"Checking original Date values in {table}...")
    try:
        # Step 1: Inspect the Date column format
        query = f"SELECT DISTINCT Date FROM {table} LIMIT 5;"
        dates = pd.read_sql(query, conn)
        print(f"Sample Date values in {table}:\n", dates)

        # Step 2: Convert or standardize the Date format if necessary
        if len(str(dates.iloc[0, 0])) == 4:  # If the Date is only YYYY
            print(f"No conversion needed for {table}, Date is already in YEAR format.")
        else:
            # Convert numeric YYYYMMDD to YYYY-MM-DD
            convert_query = f"""
            CREATE TABLE {table}_temp AS
            SELECT
                CAST(SUBSTR(Date, 1, 4) || '-' || SUBSTR(Date, 5, 2) || '-' || SUBSTR(Date, 7, 2) AS DATE) AS Date,
                Close, High, Low, Open, Volume, Asset, Type
            FROM {table};
            """
            conn.execute(convert_query)
            conn.execute(f"DROP TABLE {table};")
            conn.execute(f"ALTER TABLE {table}_temp RENAME TO {table};")
            print(f"Date column in {table} converted to DATE format.")
    except Exception as e:
        print(f"Error processing Date column in {table}: {e}")

# Close the connection
conn.close()



Checking original Date values in solar_data...
Sample Date values in solar_data:
          Date
0  2015-03-27
1  2015-03-30
2  2015-03-31
3  2015-04-01
4  2015-04-02
Date column in solar_data converted to DATE format.
Checking original Date values in indicators_data...
Sample Date values in indicators_data:
          Date
0  2015-01-02
1  2015-01-05
2  2015-01-06
3  2015-01-07
4  2015-01-08
Date column in indicators_data converted to DATE format.
Checking original Date values in oil_data...
Sample Date values in oil_data:
          Date
0  2015-01-02
1  2015-01-05
2  2015-01-06
3  2015-01-07
4  2015-01-08
Date column in oil_data converted to DATE format.
Checking original Date values in wind_data...
Sample Date values in wind_data:
          Date
0  2015-01-02
1  2015-01-05
2  2015-01-06
3  2015-01-07
4  2015-01-08
Date column in wind_data converted to DATE format.


In [50]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
db_path = "financial_data.db"
conn = sqlite3.connect(db_path)

# Query to list all tables
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(tables_query, conn)

print("Available tables in the database:")
print(tables)

# Loop through each table and fetch its schema
print("\nSchema for each table:")
for table in tables['name']:
    print(f"\nSchema for table: {table}")
    schema_query = f"PRAGMA table_info({table});"
    schema = pd.read_sql(schema_query, conn)
    print(schema)

# Close the connection
conn.close()

Available tables in the database:
                  name
0  combined_statistics
1           solar_data
2      indicators_data
3             oil_data
4            wind_data

Schema for each table:

Schema for table: combined_statistics
    cid             name  type  notnull dflt_value  pk
0     0       Unnamed: 0  TEXT        0       None   0
1     1             Mean  REAL        0       None   0
2     2           Median  REAL        0       None   0
3     3          Std Dev  REAL        0       None   0
4     4              Min  REAL        0       None   0
5     5              Max  REAL        0       None   0
6     6  25th Percentile  REAL        0       None   0
7     7  75th Percentile  REAL        0       None   0
8     8         Skewness  REAL        0       None   0
9     9         Kurtosis  REAL        0       None   0
10   10          Company  TEXT        0       None   0

Schema for table: solar_data
   cid    name  type  notnull dflt_value  pk
0    0    Date   NUM        0 

# Creating Datasets in Excel with Multiple Sheets for The Tableau

In [51]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
db_path = "financial_data.db"
conn = sqlite3.connect(db_path)

# List of tables to export to Excel
tables_to_export = ["solar_data", "indicators_data", "oil_data", "wind_data"]

# Output Excel file
output_excel_path = "Financial_Data.xlsx"

# Create an Excel writer
with pd.ExcelWriter(output_excel_path) as writer:
    for table in tables_to_export:
        print(f"Exporting {table} to Excel sheet...")
        # Read data from the table
        query = f"SELECT * FROM {table};"
        data = pd.read_sql(query, conn)
        # Write data to a sheet in the Excel file
        data.to_excel(writer, sheet_name=table, index=False)

# Close the connection
conn.close()
print(f"Excel file created: {output_excel_path}")


Exporting solar_data to Excel sheet...
Exporting indicators_data to Excel sheet...
Exporting oil_data to Excel sheet...
Exporting wind_data to Excel sheet...
Excel file created: Financial_Data.xlsx
